In [1]:
from tqdm import tqdm_notebook as tqdm
from scipy.spatial import distance
from datetime import datetime
from math import cos, sin

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import glob
import math
import cv2
import abc
import os


# Image Behavior Definition

In [2]:
class Dental_Crop:
        
        @staticmethod
        def polygon(image, points):
                regular_point = points - points.min(axis=0)
                rect = cv2.boundingRect(points)
                x, y, w, h = rect
                croped = image[y:y+h, x:x+w].copy() 
                mask = np.zeros(croped.shape, np.uint8)
                cv2.drawContours(mask, [regular_point], -1, (255, 255, 255), -1, cv2.LINE_AA)
                polygon_tooth = cv2.bitwise_and(croped, croped, mask=mask)
                return polygon_tooth
        
        
        @staticmethod
        def quarter_polygon(image, points, right=True):
                regular_point = points - points.min(axis=0)
                
                if right == True:
                    points = get_quarter_points(points, right)
                
                rect = cv2.boundingRect(points)
                x, y, w, h = rect
                croped = image[y:y+h, x:x+w].copy() 
                mask = np.zeros(croped.shape, np.uint8)
                cv2.drawContours(mask, [regular_point], -1, (255, 255, 255), -1, cv2.LINE_AA)
                polygon_tooth = cv2.bitwise_and(croped, croped, mask=mask)
                return polygon_tooth
        
        @staticmethod
        def rectangle(image, points):
                rect = cv2.boundingRect(points)
                x, y, w, h = rect
                croped = image[y:y+h, x:x+w].copy() 
                return croped

In [3]:
class Dental_Rotation:
        
        @staticmethod
        def rotate(image, points, image_center, rotate_angle, max_rect=True):
                rotate_image = rotate(image, rotate_angle, image_center, max_rect)
                return rotate_image
            
        @staticmethod
        def centralize(image, points, max_rect=True):
                rotate_angle = get_rotate_degree(image, points)
                rotate_image = rotate(image, rotate_angle, max_rect)
                return rotate_image


In [4]:
class border_preprocessing:
       
        @staticmethod
        def trim_border(image):
                if not np.sum(image[0]):     
                        return border_preprocessing.trim_border(image[1:])

                elif not np.sum(image[-1]):  
                        return border_preprocessing.trim_border(image[:-2])

                elif not np.sum(image[:,0]):
                        return border_preprocessing.trim_border(image[:,1:]) 

                elif not np.sum(image[:,-1]):
                        return border_preprocessing.trim_border(image[:,:-2])    
                return image
        
        @staticmethod
        def padding(image, padding_height = 700, padding_width = 400):
                mask_size = (padding_height, padding_width)
                tooth_h, tooth_w = image.shape
                mask = np.zeros(mask_size)
                yoff = round((mask_size[0]-tooth_h)/2)
                xoff = round((mask_size[1]-tooth_w)/2)
                result = mask.copy()
                result[yoff:yoff+tooth_h, xoff:xoff+tooth_w] = image
                return result
            

In [5]:
def init_directory(directory): 
        if not os.path.exists(directory):
                os.makedirs(directory)
def pd_dict_2_pd_dataframe(pd_json):
        table = pd.DataFrame(columns=["id", "left", "right"])

        for key in pd_json:
                front = pd_json[key]["lingual"]["CAL"]
                back  = pd_json[key]["buccal"]["CAL"]
                left  = min(front[0], back[0])
                right = min(front[2], back[2])
                table = table.append({
                        "id": key,
                        "left": left,
                        "right": right
                }, ignore_index=True)

        table.set_index("id" , inplace=True) 
        return table

def get_table_file(patient_id):
        table_infos = [ i for i in glob.iglob("Data/*/*.json")]
        for table_name in table_infos:
                if patient_id in table_name:
                        pd_json   = json.load(open(table_name, "r"))
                        pd_table = pd_dict_2_pd_dataframe(pd_json)
                        return pd_table
                    
def get_PD_pair(pd_table, label_name):
        
        number, side = label_name.split("_") 
        result = pd_table.loc[[number]].values[0]
        
        if side == "LR":
                return (result[0], result[1])
        elif side == "L" or side == "LH" or side == "HL": 
                return (result[0], -99)
        elif side == "R" or side == "RH" or side == "HR":
                return (-99, result[1])

def rotate(image, angle, image_center, max_bound_rect=True):
        
        height, width = image.shape[:2] # image shape has 3 dimensions
        rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1)
        
        if not max_bound_rect:
                rotated_image = cv2.warpAffine(image, rotation_mat, (width, height))
                return rotated_image
            
        # rotation calculates the cos and sin, taking absolutes of those.
        abs_cos = abs(rotation_mat[0,0]) 
        abs_sin = abs(rotation_mat[0,1])

        # find the new width and height bounds
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)

        rotation_mat[0, 2] += bound_w/2 - image_center[0]
        rotation_mat[1, 2] += bound_h/2 - image_center[1]

        # rotate image with the new bounds and translated rotation matrix
        rotated_image = cv2.warpAffine(image, rotation_mat, (bound_w, bound_h))
        return rotated_image

def recognize_line(line):
        sort_up_and_down = lambda line: [line[1], line[0]] if line[1][1] < line[0][1] else line
        check_clockwise  = lambda line: True if line[0][0] > line[1][0] else False    # up >>>> down
        line = sort_up_and_down(line)
        clockwise = check_clockwise(line)
        return line, clockwise
    
def get_line_angle(line, clockwise):
        vector = line[1] - line[0]
        length = np.linalg.norm(vector)
        cos    = np.dot(vector, [0, 1]) / length
        angle = math.acos(cos) *180 / math.pi
        return angle if clockwise == True else -angle

def get_rotate_degree(points):
        points = sorted(points ,key=lambda point:point[1])
        up_center_point   = [(points[0][0]+points[1][0])/2,(points[0][1]+points[1][1])/2]
        down_center_point = [(points[2][0]+points[3][0])/2,(points[2][1]+points[3][1])/2]
        midline = [up_center_point,down_center_point]
        midline = np.array(midline)
        midline , midline_direction = recognize_line(midline)
        midline_angle = get_line_angle(midline, midline_direction)
        
        clock = lambda x: "clockwise" if x == True else "counterclockwise"
        return midline_angle

In [6]:
class Image_Pipeline:
        def __init__(self, output_dir, input_dir, **kwargs):
                self.output_dir = output_dir
                self.input_dir  = input_dir
                
                for name, value in kwargs.items():
                    setattr(self, name, value)
                
        def run(self):
            raise NotImplementedError

# CAL Pipeline ( To Do : Table to Dict , and need to decouple ! )

In [7]:
class CAL_Pipeline(Image_Pipeline):
        def __init__(self, output_dir, input_dir, **kwargs):
                super(CAL_Pipeline, self).__init__(output_dir, input_dir, **kwargs) 
                if not hasattr(self, "rotate_range"):
                        raise AttributeError
                
                if not hasattr(self, "enhance"):
                        raise AttributeError
        
        def run(self):
                jsons = [ i for i in glob.iglob("%s/*/*/*.json" % self.input_dir)]
                images = [ i.replace("json", "png") for i in jsons]

                mapping_dict = {}
                no_table, no_side = set(), set()
                init_directory(self.output_dir)
                
                
                for data, image in tqdm(zip(jsons, images)):
                        tooth_img  = cv2.imread(image, 0)
                        tooth_data = json.load(open(data, "r"))["shapes"]
                        patient_id = image.split("\\")[1]
                        pd_table   = get_table_file(patient_id)
            
                        if pd_table is None:
                                no_table.add(patient_id)
                                continue

                        for tooth in tooth_data:
                                pd_pair  = get_PD_pair(pd_table, tooth["label"])
                                points   = np.array(tooth["points"]).astype(int)
                                points   = np.where(points < 0, 0, points)
                                
                                rect = cv2.boundingRect(points)
                                x, y, w, h = rect
                                mask = np.zeros(tooth_img.shape, np.uint8)
                                mask[y:y+h, x:x+w] = 255
                                rotate_center = ( (2*x+w)//2, (2*y+h)//2 )
                                
                                zero_degree = int(get_rotate_degree(points))
                                deg_lowbound, deg_upperbound, step = self.rotate_range
                                
                                deg_lowbound -= zero_degree
                                deg_upperbound -= zero_degree
                                
                                for idx, degree in enumerate(range(deg_lowbound, deg_upperbound, step), 1):
                                        rect = cv2.boundingRect(points)
                                        
                                        ro_mask = Dental_Rotation.rotate(mask, points, rotate_center, degree, max_rect=False)
                                        ro_tooth = cv2.bitwise_and(tooth_img, tooth_img, mask=ro_mask)
                                        ro_tooth = rotate(ro_tooth, -degree, rotate_center)
                                        ro_tooth = border_preprocessing.trim_border(ro_tooth)
                                        
                                        tooth_h, tooth_w = ro_tooth.shape
                                        
                                        if pd_pair == None:
                                                no_side.add(data)
                                                continue
                                        
                                        if self.enhance:
                                                ro_tooth = cv2.equalizeHist(ro_tooth.astype("uint8"))
                                        
                                        number, side = tooth["label"].split("_")

                                        if int(number) <= 16:
                                                ro_tooth = cv2.flip(ro_tooth, 0)

                                        filename = datetime.utcnow().isoformat(sep='-', timespec='milliseconds').replace(".", "-").replace(":", "-")[-12:]
                                        filename = '%s/%s_%s_%s_%d.png' % (self.output_dir, filename, image.split("\\")[2], number, degree + zero_degree)   
                                        
                                        mapping_dict[filename] = pd_pair
                                        
                                        cv2.imwrite(filename, ro_tooth)
                                    
                json.dump(mapping_dict, open("%s/mapping.json" % self.output_dir, 'w'), indent=4)

                with open("%s/loss_table" % self.output_dir, "w") as f:
                        no_table = list(no_table)
                        no_table = "\n".join(no_table)
                        f.write(no_table)

                with open("%s/loss_side" % self.output_dir, "w") as f:
                        no_side = list(no_side)
                        no_side = "\n".join(no_side)
                        f.write(no_side)


# PBL Pipeline

In [8]:
class PBL_situation:
        def __init__(self, PBLTable_path, DataPath):
                self.PBLTable_path = PBLTable_path
                self.DataPath      = DataPath
                
        def clean_invalid_value(self):
                self.PBL_Table = pd.read_csv(self.PBLTable_path, sep=",")
                dir_name = [ "%s/" % self.DataPath + i.split(" ")[0] for i in list(self.PBL_Table["sub_dir_name"]) ]
                self.PBL_Table["dir_name"] = dir_name
                self.PBL_Table = self.PBL_Table[["dir_name", "sub_dir_name", "image_name", "tooth_num", "state"]]
                self.PBL_Table = self.PBL_Table[~self.PBL_Table["tooth_num"].isnull() & ~self.PBL_Table["state"].isnull()]
                self.PBL_Table = self.PBL_Table.astype("str")
                self.PBL_Table = self.PBL_Table.replace( {'state' : { '2': -999 }} )
                self.PBL_Table = self.PBL_Table.replace( {'state' : { 'III' : 3, 'II' : 2, 'I' : 1 }} )
                self.PBL_Table = self.PBL_Table.replace( {'state' : { 'iii' : 3, 'ii' : 2, 'i' : 1 }} )
                self._PBL_directory = np.unique(self.PBL_Table["dir_name"])
                return self
            
        def MD_transfer_LR(self):
                pathes = []
                for idx, row in self.PBL_Table.iterrows():
                        tooth_num = row["tooth_num"][:-1]
                        if not tooth_num.isdigit():
                                self.PBL_Table.drop(idx, inplace=True)
                                continue
                        else:

                                if  9 <= int(tooth_num) <= 24: # L--> M, R-->D
                                        if "M" in row["tooth_num"]:
                                                self.PBL_Table.loc[ idx ,"tooth_num"] = tooth_num + "L"
                                        if "D" in row["tooth_num"]:
                                                self.PBL_Table.loc[ idx ,"tooth_num"] = tooth_num + "R"

                                elif 1 <= int(tooth_num) <= 8 or 24 < int(tooth_num) <= 32:
                                        if "M" in row["tooth_num"]:
                                                self.PBL_Table.loc[ idx ,"tooth_num"] = tooth_num + "R"
                                        if "D" in row["tooth_num"]:
                                                self.PBL_Table.loc[ idx ,"tooth_num"] = tooth_num + "L"

                                else:
                                        print(row)
                                        raise ValueError

                        path = "%s/%s/%s.PNG" % (row["dir_name"], row["sub_dir_name"], row["image_name"])
                        pathes.append(path)

                self.PBL_Table["path"] = pathes
                
                self._Simple_PBL_Table = self.PBL_Table.loc[:, ["path", "tooth_num", "state"]]
                self._Simple_PBL_Table = self._Simple_PBL_Table[~self._Simple_PBL_Table["tooth_num"].isnull()]
                return self
        
        def table_2_dict(self):
                image_groups = self._Simple_PBL_Table.groupby("path")
                self._label_dict = {}

                for group in image_groups.groups:
                        if group not in self._label_dict:
                                self._label_dict[group] = {}

                        for idx, item in image_groups.get_group(group).iterrows():
                                tooth_num = item["tooth_num"][:-1]
                                if tooth_num not in self._label_dict[group]:
                                        self._label_dict[group][tooth_num] = [-99, -99]

                                if item["tooth_num"][-1] == "L":
                                        self._label_dict[group][tooth_num][0] = item["state"]

                                elif item["tooth_num"][-1] == "R":
                                        self._label_dict[group][tooth_num][1] = item["state"]

                                else:
                                        print(item["tooth_num"])
                                        raise ValueError 
                return self 
        
        def run(self):
                self.clean_invalid_value().MD_transfer_LR().table_2_dict()
                return self

            
        @property
        def label_dict(self):
                return self._label_dict
        
        @property
        def Simple_PBL_Table(self):
                return self._Simple_PBL_Table
        
        @property
        def PBL_directory(self):
                return self._PBL_directory
            

In [9]:
class PBL_Pipeline(Image_Pipeline):
        def __init__(self, output_dir, input_dir, **kwargs):
                super(PBL_Pipeline, self).__init__(output_dir, input_dir, **kwargs) 
                if not hasattr(self, "rotate_range"):
                        raise AttributeError
                
                if not hasattr(self, "enhance"):
                        raise AttributeError
        
        def run(self):
                PBL_Info = PBL_situation("PBL_Table.csv", "Label").run()
                
                jsons = [ i for directory in PBL_Info.PBL_directory for i in glob.iglob("%s/*/*.json" % directory) ]
                images = [ i.replace("json", "PNG").replace("\\", "/") for i in jsons ]
    
                mapping_dict = {}
                no_table, no_side = set(), set()
                init_directory(self.output_dir)
                
                
                for data, image in tqdm(zip(jsons, images)):
                        tooth_img  = cv2.imread(image, 0)
                        tooth_data = json.load(open(data, "r"))["shapes"]
                        
            
                        if image not in PBL_Info.label_dict:
                                no_table.add(image)
                                continue

                        for tooth in tooth_data:
                                points   = np.array(tooth["points"]).astype(int)
                                points   = np.where(points < 0, 0, points)
                                rect = cv2.boundingRect(points)
                                x, y, w, h = rect
                                mask = np.zeros(tooth_img.shape, np.uint8)
                                mask[y:y+h, x:x+w] = 255
                                rotate_center = ( (2*x+w)//2, (2*y+h)//2 )
                                
                                zero_degree = int(get_rotate_degree(points))
                                deg_lowbound, deg_upperbound, step = self.rotate_range
                                
                                deg_lowbound -= zero_degree
                                deg_upperbound -= zero_degree
                                
                                for idx, degree in enumerate(range(deg_lowbound, deg_upperbound, step), 1):
                                        rect = cv2.boundingRect(points)
                                        
                                        ro_mask = Dental_Rotation.rotate(mask, points, rotate_center, degree, max_rect=False)
                                        ro_tooth = cv2.bitwise_and(tooth_img, tooth_img, mask=ro_mask)
                                        ro_tooth = rotate(ro_tooth, -degree, rotate_center)
                                        ro_tooth = border_preprocessing.trim_border(ro_tooth)
                                        
                                        tooth_h, tooth_w = ro_tooth.shape
                                        number, side = tooth["label"].split("_")
                                        
                                        if number not in PBL_Info.label_dict[image]:
                                                no_side.add(data)
                                                continue
                                                
                                        if self.enhance:
                                                ro_tooth = cv2.equalizeHist(ro_tooth.astype("uint8"))

                                        if int(number) <= 16:
                                                ro_tooth = cv2.flip(ro_tooth, 0)

                                        filename = datetime.utcnow().isoformat(sep='-', timespec='milliseconds').replace(".", "-").replace(":", "-")[-12:]
                                        filename = '%s/%s_%s_%s_%d.png' % (self.output_dir, filename, image.split("/")[2], number, degree + zero_degree)   
                                        
                                        mapping_dict[filename] = PBL_Info.label_dict[image][number]
                                        cv2.imwrite(filename, ro_tooth)
                                    
                json.dump(mapping_dict, open("%s/mapping.json" % self.output_dir, 'w'), indent=4)

                with open("%s/loss_table" % self.output_dir, "w") as f:
                        no_table = list(no_table)
                        no_table = "\n".join(no_table)
                        f.write(no_table)

                with open("%s/loss_side" % self.output_dir, "w") as f:
                        no_side = list(no_side)
                        no_side = "\n".join(no_side)
                        f.write(no_side)


# Generating Data

In [10]:
parameters = [  
                
                {
                        "output_dir"  : "Dental_Data/PBL/10",
                        "input_dir"   : "Label",
                        "rotate_range": (-10, 10, 1),
                        "enhance"     : False
                },
            
    
                {
                        "output_dir"  : "Dental_Data/PBL/10_enhance",
                        "input_dir"   : "Label",
                        "rotate_range": (-10, 10, 1),
                        "enhance"     : True
                },
                
                {
                        "output_dir"  : "Dental_Data/CAL/10",
                        "input_dir"   : "Label",
                        "rotate_range": (-10, 10, 1),
                        "enhance"     : False
                },
            
    
                {
                        "output_dir"  : "Dental_Data/CAL/10_enhance",
                        "input_dir"   : "Label",
                        "rotate_range": (-10, 10, 1),
                        "enhance"     : True
                },
    
]
    
for parameter in parameters:
    
        if "PBL" in parameter["output_dir"]:
                PBL_Pipeline(**parameter).run()
                
        if "CAL" in parameter["output_dir"]:
                CAL_Pipeline(**parameter).run()